# Escenario 2.1 - Alice y Bob con clave

En este escenario se cubre el caso en el que Alice cifra para Bob y Bob descifra los mensajes por medio de una clave. Sin embargo, se ha añadido un agente malicioso, Eve, que quiere interceptar la comunicación.

Se lleva a cabo un entrenamiento, una evaluación de los resultados y se dibuja una gráfica para ilustrarlos. Como se hacen muchas ejecuciones, se grafica el resultado obtenido en cada ejecución. Hay cierta variabilidad en los resultados al ser los pesos de las redes neuronales aleatorios, así como los mensajes generados en el entrenamiento y la evaluación.

Avisar del tiempo, una ejecución con las _epochs_ definidas por defecto (de 300 en 300 hasta llegar a 6000), puede tardar el triple que en los escenarios 1 debido al aumento en al complejidad del escenario.

## Imports

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

from data_utils import generar_mensajes

import numpy as np
import time as t

## Las redes neuronales

En esta sección definimos las redes a utilizar más adelante.

**Alice**: Red neuronal que cifra los mensajes.
1. Recibe como parámetro el número de bits de los mensajes que va a cifrar y entonces define la forma de entrada.
2. Concatena el mensaje de entrada con al clave y eso es lo que se le pasa a la red, para cifrar así el mensaje.
3. final_input va a recibir algo de la forma mensaje + clave, por lo que esta vez recibe una dupla.
4. Va a tener dos capas, con 128 y 64 neuronas, que reciben la entrada de la anterior y usan la función de activación relu. _Dense_ quiere decir que la capa está completamente conectada con la anterior.

La última capa tiene una función de activación lineal porque luego la va a procesar Bob. No es necesario que sea binaria todavía. Tiene también tantas neuronas como bits, porque se busca que saque el mensaje cifrado y, por último, la función kernel_regularizer ayuda a evitar el sobreajuste penalizando las capas, de forma a que afecta la función de pérdida/coste.

In [ ]:
def crear_modelo_alice(bits):
    input_msg = Input(shape=(bits,), name='mensaje_original')

    input_key = Input(shape=(bits,), name='clave_simetrica')
    x = Concatenate()([input_msg, input_key])
    final_input = [input_msg, input_key]

    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    cifrado = Dense(bits, activation='linear', kernel_regularizer=l2(0.01))(x)
    return Model(final_input, cifrado, name='Alice')

**Bob**: Red neuronal que descifra los mensajes.
Funciona parecido a Alice, salvo que tiene capas de 64 y 64 neuronas. 
Lo más destacable es que la función de activación de la última capa es, en este caso, la función sigmoide. La función sigmoide devuelve un valor entre 0 y 1 y esos son los valores que luego se procesan para inferir si la predicción es el valor 0 o el valor 1 en la reconstrucción.

Como siempre, el mensaje cifrado y la clave se concatenan para llevar a cabo el descifrado.

En este caso, cabe mencionar que, para evitar sobreajuste, se ha añadido un _dropout_.

In [ ]:
def crear_modelo_bob(bits):
    input_cifrado = Input(shape=(bits,), name='mensaje_cifrado')
    input_key = Input(shape=(bits,), name='clave_simetrica')
    x = Concatenate()([input_cifrado, input_key])
    final_input = [input_cifrado, input_key]
    
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    reconstruido = Dense(bits, activation='sigmoid')(x)
    return Model(final_input, reconstruido, name='Bob')

**Eve**: Red neuronal que pretende interceptar los mensajes.
Funciona como Bob, salvo que no recibe niguna clave, solo el mensaje cifrado.

In [ ]:
def crear_modelo_eve(bits):
    input_cifrado = Input(shape=(bits,), name=f'mensaje_cifrado_eve')
    x = Dense(64, activation='relu')(input_cifrado)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    salida = Dense(bits, activation='sigmoid', name=f'mensaje_reconstruido_eve')(x)
    return Model (input_cifrado, salida, name='Eve')